#Introduction

####This project demonstrates my proficiency as a Data Science student by deploying a machine learning model to predict weight based on height. Two approaches were explored: Gradio and Flask with ngrok. Through a user-friendly interface and comprehensive documentation, this project showcases practical deployment skills. Let's explore the application of machine learning in real-world scenarios together!

In [43]:
# Importing the Google Drive library to mount the drive and access files
from google.colab import drive
drive.mount("/drive")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [44]:
cd /drive/My Drive/DataSet

/drive/My Drive/DataSet


In [10]:
# Installing the Gradio library
!pip install gradio

In [11]:
import gradio as gr
import numpy as np
import pickle

# Carregar o modelo usando o pickle
model = pickle.load(open('pipeline_with_model.pkl', 'rb'))

# Função de previsão para o modelo
def predict_weight(casado, possui_carro, possui_imovel, ensino_superior, salario, idade):
    # Converter as entradas em uma matriz bidimensional (2D) usando numpy reshape
    input_data = np.array([casado, possui_carro, possui_imovel, ensino_superior, salario, idade]).reshape(1, -1)

    # Realizar a previsão usando o modelo
    prediction = model.predict(input_data)

    # Retornar a previsão como uma string
    return 'Status final: {}'.format(prediction[0])

# Criar a interface Gradio para o modelo
iface = gr.Interface(
    fn=predict_weight,
    inputs=[gr.inputs.Number(label="Casado"),
            gr.inputs.Number(label="Possui Carro"),
            gr.inputs.Number(label="Possui Imóvel"),
            gr.inputs.Number(label="Ensino Superior"),
            gr.inputs.Number(label="Salário"),
            gr.inputs.Number(label="Idade")],  # Campos de entrada do formulário
    outputs="text",  # Campo de saída para exibir o resultado da previsão
    layout="vertical",  # Organizar elementos verticalmente na interface
    live=True  # Atualizar a saída conforme o usuário interage com a entrada (opcional)
)

# Iniciar a interface Gradio
iface.launch(share=True)



<ipython-input-11-7f02e5690947>:22: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Number(label="Casado"),
<ipython-input-11-7f02e5690947>:22: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Number(label="Casado"),
<ipython-input-11-7f02e5690947>:23: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Number(label="Possui Carro"),
<ipython-input-11-7f02e5690947>:23: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Number(label="Possui Carro"),
<ipython-input-11-7f02e5690947>:24: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3ac106ce2500853c16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
with open('pipeline_with_model.pkl', 'rb') as f:
    data = f.read()
    print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#Using Flask with ngrok for Model Deployment

In [13]:
# Installing pyngrok to set up the ngrok tunnel
!pip install pyngrok


In [45]:
# Importing the Flask and pyngrok libraries for the second approach
from flask import Flask, render_template, request
from pyngrok import ngrok
import numpy as np
import pickle

# Loading the model again (not necessary if already loaded)
model = pickle.load(open('pipeline_with_model.pkl', 'rb'))

# Setting up the ngrok tunnel to access the Flask application
port_no = 5000
ngrok.set_auth_token("2SiTmGVNXML31sJ7lnjvUZ5CZk2_i78e3DCTjft99rcqjWAt")
public_url = ngrok.connect(port_no).public_url

# Creating the Flask application
app = Flask(__name__, template_folder="templates")

# Defining the root route of the application
@app.route("/")
def home():
    return render_template('deploy.html')

# Printing the public link to access the Flask application
print(f"To access the Global link please click {public_url}")


# Defining the route to make predictions
@app.route('/getprediction', methods=['POST'])
def getprediction():
    try:
        print("Request form values:", request.form.values())  # Adiciona essa linha para verificar as entradas recebidas
        input_data = [float(x) for x in request.form.values()]
        final_input = np.array(input_data).reshape(1, -1)
        prediction = model.predict(final_input)
        print("Prediction:", prediction)  # Adiciona essa linha para verificar o valor da previsão
        return render_template('deploy.html', output=prediction[0])
    except Exception as e:
        return render_template('deploy.html', output="Valor inválido")


# Starting the Flask server
if __name__ == '__main__':
    app.run(debug=False, port=port_no)



To access the Global link please click https://c6ff-35-204-47-124.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 21:13:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 21:13:38] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 21:13:43] "POST /getprediction HTTP/1.1" 200 -


Request form values: <generator object MultiDict.values at 0x7cfaf82034c0>
Prediction: [0]
